In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
import visuals as vs

In [28]:
data = pd.read_csv('housing.csv')
prices = data['MEDV']
features = data.drop('MEDV', axis = 1)
print("Набор данных о стоимости жилья в бостоне имеет {} образцов с {} признаками каждый".format(*data.shape))

Набор данных о стоимости жилья в бостоне имеет 489 образцов с 4 признаками каждый


### Немного статистики 

In [29]:
# Minimum price of the data
minimum_price = np.amin(prices)
maximum_price = np.amax(prices)
mean_price = np.mean(prices)
median_price = np.median(prices)
std_price = np.std(prices)

# Show the calculated statistics
print("Статистика:\n")
print("Минимальная стоимость: ${}".format(minimum_price)) 
print("Максимальная стоимость : ${}".format(maximum_price))
print("Средняя стоимость : ${}".format(mean_price))
print("Медианная стоимость ${}".format(median_price))
print("Стандартное отклонение стоимости: ${}".format(std_price))

Статистика:

Минимальная стоимость: $105000.0
Максимальная стоимость : $1024800.0
Средняя стоимость : $454342.9447852761
Медианная стоимость $438900.0
Стандартное отклонение стоимости: $165171.13154429474


R2 оценка она же коэффициент детерминации 
Коэффициент детерминации ( R 2 {\displaystyle R^{2}} R^2 — R-квадрат) — это доля дисперсии зависимой переменной, объясняемая рассматриваемой моделью зависимости, то есть объясняющими переменными. Более точно — это единица минус доля необъяснённой дисперсии (дисперсии случайной ошибки модели, или условной по факторам дисперсии зависимой переменной) в дисперсии зависимой переменной. Его рассматривают как универсальную меру зависимости одной случайной величины от множества других. В частном случае линейной зависимости R 2 {\displaystyle R^{2}} R^2 является квадратом так называемого множественного коэффициента корреляции между зависимой переменной и объясняющими переменными. В частности, для модели парной линейной регрессии коэффициент детерминации равен квадрату обычного коэффициента корреляции между y и x. 

Наилучшая возможная оценка - 1,0, и она может быть отрицательной (поскольку модель может быть произвольно хуже). Постоянная модель, которая всегда прогнозирует ожидаемое значение y, независимо от входных характеристик, получит оценку R ^ 2 0,0.

In [15]:
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):    
    score = r2_score(y_true, y_predict)    
    return score

###  Разделение данных 

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, prices, test_size=0.2, random_state = 42)

### Кросс Валидация

In [31]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

def fit_model(X, y):
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)

    regressor = DecisionTreeRegressor()

    params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

    scoring_fnc = make_scorer(performance_metric)
    
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

### ОБучение

In [33]:
reg = fit_model(X_train, y_train)
print("Максимальная глубина {} оптимальна для модели".format(reg.get_params()['max_depth']))

Максимальная глубина 4 оптимальна для модели


In [34]:
client_data = [[5, 17, 15], # Client 1
               [4, 32, 22], # Client 2
               [8, 3, 12]]  # Client 3

# Show predictions
for i, price in enumerate(reg.predict(client_data)):
    print("Предполагаемая стоимость {}'s home: ${:,.2f}".format(i+1, price))

Предполагаемая стоимость 1's home: $403,025.00
Предполагаемая стоимость 2's home: $237,478.72
Предполагаемая стоимость 3's home: $931,636.36


In [35]:
vs.PredictTrials(features, prices, fit_model, client_data)

Trial 1: $391,183.33
Trial 2: $419,700.00
Trial 3: $415,800.00
Trial 4: $420,622.22
Trial 5: $413,334.78
Trial 6: $411,931.58
Trial 7: $399,663.16
Trial 8: $407,232.00
Trial 9: $351,577.61
Trial 10: $413,700.00

Range in prices: $69,044.61
